In [1]:
from huggingface_hub import login

login(token="")

/home/karthick/finetuning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/karthick/.cache/huggingface/token
Login successful


In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
tokenizer1 = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import transformers
import torch

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards: 100%|███████████████████████████████████████████| 4/4 [00:04<00:00,  1.19s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [6]:
import transformers
from datasets import load_dataset
from typing import Dict, List, Optional

dataset_name = "tryhighlight/task_dataset"
#Importing the dataset
input_dataset = load_dataset(dataset_name, split="train")
print (input_dataset)

Dataset({
    features: ['NAME', 'CONVERSATION', 'TASK'],
    num_rows: 30200
})


In [7]:
system_prompt = "The user will provide his/her full name followed by an email thread from their inbox that contains one or more email conversations. Looking at the conversation, detect if there are any TODOs that the user has to complete as a result of the conversation. If yes, just provide the short single line task that can be directly added to the todo list. If there is no task detected as a TODO, just output the exact phrase \"No task\". If the conversation is about a promotional or advertisement related, please output \"No task\". If the conversation is directed or addressed to someone else, then output \"No task\". Just provide the short single line task or the phrase \"No task\", no other explanation is needed."

In [8]:
max_len = 512

In [10]:
def check_length(row):
    # This function checks if the tokenized length is within the max length allowed
    input_content = tokenizer1.encode(system_prompt + row["NAME"] + row["CONVERSATION"] + row["TASK"],
                                        add_special_tokens=True,
                                        truncation=False,
                                        return_length=True,
                                        max_length=None)
    return len(input_content) <= max_len - 20 # extra 20 tokens for the chat template

# Filter the dataset to exclude entries that are too long
input_dataset = input_dataset.filter(check_length, num_proc=4)
print(f"Size of the dataset after filtering: {len(input_dataset)} samples")

Filter (num_proc=4): 100%|████████████████████████████████| 30200/30200 [00:09<00:00, 3177.60 examples/s]


Size of the dataset after filtering: 17834 samples


In [11]:
dataset=input_dataset

In [12]:
dataset = dataset.select(range(100))

In [13]:
dataset = dataset.train_test_split(test_size=0.1, seed=42)

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['NAME', 'CONVERSATION', 'TASK'],
        num_rows: 90
    })
    test: Dataset({
        features: ['NAME', 'CONVERSATION', 'TASK'],
        num_rows: 10
    })
})

In [15]:
for row in dataset["train"]:
    messages = [ 
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": "My name is " + row["NAME"] + " \n" + row["CONVERSATION"]}
    ]
    # print (messages)
    # print ('name : ',  row["NAME"])
    # print ('CONVERSATION :', row["CONVERSATION"])
    outputs = pipeline(
        messages,
        max_new_tokens=20,
        eos_token_id=terminators,
        do_sample=True,
        temperature=1.0,
        top_p=0.9,
    )
    print ('Expected output : ', row['TASK'])
    print ('Inference output : ', outputs[0]['generated_text'][-1]["content"], '\n')

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Expected output :  Task : Approve Access Request for matt.smith@enron.com
Inference output :  Approve access request for matt.smith@enron.com 



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Expected output :  No task
Inference output :  No task 



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Expected output :  No task
Inference output :  No task 



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Expected output :  Task : Forward phone number to Daniel McElduff or call Bo Collins at 212-299-2260
Inference output :  I cannot help you with that. 



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Expected output :  No task
Inference output :  John Arnold

No task 

Expected output :  Task : Let Jeff know if you have an interest in the three property deals he will fax.
Inference output :  Phillip Allen, No task. 

